In [15]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.singleVariant as singleVariant
import module.multiVariant as multiVariant
import numpy as np
from numba import njit,jit, float32
from tqdm import tqdm
import module.multiHistogramSparse as multiHistogramSparse
import module.ananlysisFuncion as analysisFunction
import module.singleVariantCopulaBase as CopulaBase
reload(multiHistogramSparse)
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(analysisFunction)
reload(CopulaBase)

#attribute_names=np.array(["SALT","TEMP","U","V","W"])
attribute_names=np.array(["SALT","TEMP","U","V",])
all_ensamble_data=np.empty([0,180,20,250,250])
incremental_number=180

for name in attribute_names:
    data=utilize.readRedSeaFile(name)
    data=data.reshape(1,180,20,250,250)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)





covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=20
sizeY=250
sizeX=250
minMaxBlockSize=2
isMinMax=False

conditions=np.array([[36,40],[26,30]])

print("start fit model")
with tqdm(total=3, desc="Model fitting") as pbar:
    #oursModel=multiVariant.multiDistCopula3D(all_ensamble_data,dataBlockSize,covBlockSize,binsNumber,[sizeZ,sizeY,sizeX],minMaxBlockSize,isMinMax)
    oursModel=multiVariant.multiDistCopula3D.load(f"RedSea_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock5")
    #conditions=np.array([[0,1e5],[3e10,5e10]])
    #oursModel.fit()
    print("ours complete fit")
    pbar.update(1)
    copulaBlockSize=2
    copulaBaseModel=CopulaBase.multiVariantCopulaBase(all_ensamble_data,copulaBlockSize)
    copulaBaseModel.fit()
    print("copula complete fit")
    pbar.update(1)
    multiHistBlockSize=2
    multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
    multiHistModel.fit()
    print("multi-hist complete fit")
   
    print("complete fit")
    pbar.update(1)

start fit model


Model fitting:  33%|███▎      | 1/3 [00:01<00:02,  1.03s/it]

ours complete fit


Model fitting:  67%|██████▋   | 2/3 [05:17<03:06, 186.66s/it]

copula complete fit


Model fitting: 100%|██████████| 3/3 [08:29<00:00, 170.00s/it]

multi-hist complete fit
complete fit


In [16]:
oursModel.getStorageSize()
counts=np.array([1.9, 4.1, 7.9, 9.6, ])
copulaBaseModel.getStorageSize(counts)
multiHistModel.getStorageSize()

nonZeros=[]
for i in range(len(multiHistModel.blocks)):
    nonZeros.append(len(multiHistModel.blocks[i].hist))

nonZeros=np.array(nonZeros)
print(f"sparse non zeros counts:{nonZeros.mean()}")

v0: 267636
v1: 561120
v2: 957624
v3: 1157856
cov:150000.0
sum:3094236.0
copula base storage:12968750.0
sparse sum: 94130475
sparse non zeros counts:120.487008
